<a href="https://colab.research.google.com/github/2015samtaylor/Disseration_Analysis/blob/main/Create_Intraday_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Packages and Mount Google Drive 

%%capture
from google.colab import drive
drive.mount('/content/drive')

!pip install alpha_vantage

In [ ]:
#@title Get Data from API 

import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime
from google.colab import drive

api_key = 'QFZF2CEU5FAI6YIW'

sym = input("What is the ticker you are looking for? ")

What is the ticker you are looking for? TGT


In [ ]:
 #@title Create Intraday Highs and Lows Dataframes

ticker = sym
date = 'year1month1'

df_1 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

date = 'year1month2'
df_2 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

date = 'year1month3'
df_3 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

date = 'year1month4'
df_4 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

# -------
df = pd.concat([df_1, df_2, df_3, df_4], ignore_index = True)
df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['Day_of_the_Week'] = df['time'].dt.day_name()

time_end = df['time'].iloc[0]
time_begin = df['time'].iloc[-1]

TB = str(time_begin)[0:10]
time_begin = TB +  str(' 6:30:00')

TE = str(time_end)[0:10]
time_end = TB +  str(' 13:00:00')

df.set_index('time')

# --------
# Function to Create Intraday DataFrames as CSVs

beginning = pd.date_range(start = time_begin, end = TE, freq = 'D')
the_end = pd.date_range(start = time_end , end = TE, freq = 'D')

beginning = beginning.to_frame(index = False)
beginning.rename(columns={0 : 't'}, inplace = True)
the_end = the_end.to_frame(index = False)
the_end.rename(columns = {0 : 'ti'}, inplace = True)

first = beginning['t'].tolist()
second = the_end['ti'].tolist()

sym = sym.upper()

drive.mount('/drive')

def go(beginning_time, end_time):
    total_time = len(beginning_time)
    i = 0
    while i < total_time:
        one_day = pd.date_range(start = beginning_time[i], end = end_time[i] , freq = '15min')
        single_day = one_day.to_frame(index = True)
        single_day.rename(columns={0 : 'time'}, inplace = True)
        combo = pd.merge(single_day, df, on = 'time', how = 'inner') 
        if not combo.empty:
            data = combo
            ind = data[['low']].idxmin()
            new = pd.DataFrame()
            record = data.iloc[ind]
            answer = pd.concat([new, record], ignore_index = True)
            answer.to_csv('/drive/My Drive/Stock_Market_CSVs/'+sym+'_low_intraday.csv', index = False, mode = 'a') 
        i += 1
        
go(first, second)

# ------
drive.mount('/drive')

def go(beginning_time, end_time):
    total_time = len(beginning_time)
    i = 0
    while i < total_time:
        one_day = pd.date_range(start = beginning_time[i], end = end_time[i] , freq = '15min')
        single_day = one_day.to_frame(index = True)
        single_day.rename(columns={0 : 'time'}, inplace = True)
        combo = pd.merge(single_day, df, on = 'time', how = 'inner') 
        if not combo.empty:
            data = combo
            ind = data[['low']].idxmax()
            new = pd.DataFrame()
            record = data.iloc[ind]
            answer = pd.concat([new, record], ignore_index = True)
            answer.to_csv('/drive/My Drive/Stock_Market_CSVs/'+sym+'_high_intraday.csv', index = False, mode = 'a') 
        i += 1
        
go(first, second)

Mounted at /drive
Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
